In [1]:
import torch
import torch_tensorrt
import numpy as np
from torchvision import transforms
from torchvision.transforms.functional import to_pil_image
from torchinfo import summary
import cv2
import matplotlib.pyplot as plt
from deepmag import dataset, viz, model, train

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

/home/rhijn/VMM-real-time/motion-magnification-master/deepmag/train.py:3: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
m = torch.load('data/models/20191202-b10-r0.1-lr0.0001-00.pt').to(device)

/usr/local/lib/python3.8/dist-packages/torch/serialization.py:888: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.8/dist-packages/torch/serialization.py:888: SourceChangeWarning: source code of class 'torch.nn.modules.padding.ReflectionPad2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.8/dist-packages/torch/serialization.py:888: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute

In [3]:
# baby
transform = transforms.Compose([
    transforms.ToTensor()
])
amplification = 20
amp_f_tensor = torch.tensor(
            [[float(amplification)]], dtype=torch.float, device=device)

for i in range(300):
    print("cuda frame", i)
    frame_a = cv2.imread('data/examples/baby_frames/baby' + '{:03d}'.format(i) + '.jpg')[...,::-1]
    frame_a = torch.unsqueeze(transform(to_pil_image(frame_a)).to(device), 0)
    frame_b = cv2.imread('data/examples/baby_frames/baby' + '{:03d}'.format(i+1) + '.jpg')[...,::-1]
    frame_b = torch.unsqueeze(transform(to_pil_image(frame_b)).to(device), 0)
    pred, _, _ = m.forward(
        frame_a.to(device),
        frame_b.to(device),
        amp_f_tensor.reshape(1, 1).to(device))
    pred = pred[0].permute(1,2,0).detach().cpu().numpy()[...,::-1]
    file_path = 'data/examples/baby_frames_cuda/baby' + '{:03d}'.format(i) + 'cuda.jpg'
    cv2.imwrite(file_path, 255*pred)

torch.cuda.empty_cache()
m_trt = torch.jit.load("model_trt_960_544.ts")
for i in range(300):
    print("trt frame", i)
    frame_a = cv2.imread('data/examples/baby_frames/baby' + '{:03d}'.format(i) + '.jpg')[...,::-1]
    frame_a = torch.unsqueeze(transform(to_pil_image(frame_a)).to(device), 0)
    frame_b = cv2.imread('data/examples/baby_frames/baby' + '{:03d}'.format(i+1) + '.jpg')[...,::-1]
    frame_b = torch.unsqueeze(transform(to_pil_image(frame_b)).to(device), 0)
    pred, _, _ = m_trt.forward(
        frame_a.to(device),
        frame_b.to(device),
        amp_f_tensor.reshape(1, 1).to(device))
    pred = pred[0].permute(1,2,0).detach().cpu().numpy()[...,::-1]
    file_path = 'data/examples/baby_frames_trt/baby' + '{:03d}'.format(i) + 'trt.jpg'
    cv2.imwrite(file_path, 255*pred)

cuda frame 0
cuda frame 1
cuda frame 2
cuda frame 3
cuda frame 4
cuda frame 5
cuda frame 6
cuda frame 7
cuda frame 8
cuda frame 9
cuda frame 10
cuda frame 11
cuda frame 12
cuda frame 13
cuda frame 14
cuda frame 15
cuda frame 16
cuda frame 17
cuda frame 18
cuda frame 19
cuda frame 20
cuda frame 21
cuda frame 22
cuda frame 23
cuda frame 24
cuda frame 25
cuda frame 26
cuda frame 27
cuda frame 28
cuda frame 29
cuda frame 30
cuda frame 31
cuda frame 32
cuda frame 33
cuda frame 34
cuda frame 35
cuda frame 36
cuda frame 37
cuda frame 38
cuda frame 39
cuda frame 40
cuda frame 41
cuda frame 42
cuda frame 43
cuda frame 44
cuda frame 45
cuda frame 46
cuda frame 47
cuda frame 48
cuda frame 49
cuda frame 50
cuda frame 51
cuda frame 52
cuda frame 53
cuda frame 54
cuda frame 55
cuda frame 56
cuda frame 57
cuda frame 58
cuda frame 59
cuda frame 60
cuda frame 61
cuda frame 62
cuda frame 63
cuda frame 64
cuda frame 65
cuda frame 66
cuda frame 67
cuda frame 68
cuda frame 69
cuda frame 70
cuda frame 71
cu

trt frame 1
trt frame 2
trt frame 3
trt frame 4
trt frame 5
trt frame 6
trt frame 7
trt frame 8
trt frame 9
trt frame 10
trt frame 11
trt frame 12
trt frame 13
trt frame 14
trt frame 15
trt frame 16
trt frame 17
trt frame 18
trt frame 19
trt frame 20
trt frame 21
trt frame 22
trt frame 23
trt frame 24
trt frame 25
trt frame 26
trt frame 27
trt frame 28
trt frame 29
trt frame 30
trt frame 31
trt frame 32
trt frame 33
trt frame 34
trt frame 35
trt frame 36
trt frame 37
trt frame 38
trt frame 39
trt frame 40
trt frame 41
trt frame 42
trt frame 43
trt frame 44
trt frame 45
trt frame 46
trt frame 47
trt frame 48
trt frame 49
trt frame 50
trt frame 51
trt frame 52
trt frame 53
trt frame 54
trt frame 55
trt frame 56
trt frame 57
trt frame 58
trt frame 59
trt frame 60
trt frame 61
trt frame 62
trt frame 63
trt frame 64
trt frame 65
trt frame 66
trt frame 67
trt frame 68
trt frame 69
trt frame 70
trt frame 71
trt frame 72
trt frame 73
trt frame 74
trt frame 75
trt frame 76
trt frame 77
trt fram

In [4]:
# candle
transform = transforms.Compose([
    transforms.ToTensor()
])
amplification = 20
amp_f_tensor = torch.tensor(
            [[float(amplification)]], dtype=torch.float, device=device)

torch.cuda.empty_cache()
for i in range(471):
    print("cuda frame", i)
    frame_a = cv2.imread('data/examples/candle_frames/candle' + '{:03d}'.format(i) + '.jpg')[...,::-1]
    frame_a = torch.unsqueeze(transform(to_pil_image(frame_a)).to(device), 0)
    frame_b = cv2.imread('data/examples/candle_frames/candle' + '{:03d}'.format(i+1) + '.jpg')[...,::-1]
    frame_b = torch.unsqueeze(transform(to_pil_image(frame_b)).to(device), 0)
    pred, _, _ = m.forward(
        frame_a.to(device),
        frame_b.to(device),
        amp_f_tensor.reshape(1, 1).to(device))
    pred = pred[0].permute(1,2,0).detach().cpu().numpy()[...,::-1]
    file_path = 'data/examples/candle_frames_cuda/candle' + '{:03d}'.format(i) + 'cuda.jpg'
    cv2.imwrite(file_path, 255*pred)

torch.cuda.empty_cache()
m_trt = torch.jit.load("model_trt_640_360.ts")
for i in range(471):
    print("trt frame", i)
    frame_a = cv2.imread('data/examples/candle_frames/candle' + '{:03d}'.format(i) + '.jpg')[...,::-1]
    frame_a = torch.unsqueeze(transform(to_pil_image(frame_a)).to(device), 0)
    frame_b = cv2.imread('data/examples/candle_frames/candle' + '{:03d}'.format(i+1) + '.jpg')[...,::-1]
    frame_b = torch.unsqueeze(transform(to_pil_image(frame_b)).to(device), 0)
    pred, _, _ = m_trt.forward(
        frame_a.to(device),
        frame_b.to(device),
        amp_f_tensor.reshape(1, 1).to(device))
    pred = pred[0].permute(1,2,0).detach().cpu().numpy()[...,::-1]
    file_path = 'data/examples/candle_frames_trt/candle' + '{:03d}'.format(i) + 'trt.jpg'
    cv2.imwrite(file_path, 255*pred)

cuda frame 0
cuda frame 1
cuda frame 2
cuda frame 3
cuda frame 4
cuda frame 5
cuda frame 6
cuda frame 7
cuda frame 8
cuda frame 9
cuda frame 10
cuda frame 11
cuda frame 12
cuda frame 13
cuda frame 14
cuda frame 15
cuda frame 16
cuda frame 17
cuda frame 18
cuda frame 19
cuda frame 20
cuda frame 21
cuda frame 22
cuda frame 23
cuda frame 24
cuda frame 25
cuda frame 26
cuda frame 27
cuda frame 28
cuda frame 29
cuda frame 30
cuda frame 31
cuda frame 32
cuda frame 33
cuda frame 34
cuda frame 35
cuda frame 36
cuda frame 37
cuda frame 38
cuda frame 39
cuda frame 40
cuda frame 41
cuda frame 42
cuda frame 43
cuda frame 44
cuda frame 45
cuda frame 46
cuda frame 47
cuda frame 48
cuda frame 49
cuda frame 50
cuda frame 51
cuda frame 52
cuda frame 53
cuda frame 54
cuda frame 55
cuda frame 56
cuda frame 57
cuda frame 58
cuda frame 59
cuda frame 60
cuda frame 61
cuda frame 62
cuda frame 63
cuda frame 64
cuda frame 65
cuda frame 66
cuda frame 67
cuda frame 68
cuda frame 69
cuda frame 70
cuda frame 71
cu

trt frame 0
trt frame 1
trt frame 2
trt frame 3
trt frame 4
trt frame 5
trt frame 6
trt frame 7
trt frame 8
trt frame 9
trt frame 10
trt frame 11
trt frame 12
trt frame 13
trt frame 14
trt frame 15
trt frame 16
trt frame 17
trt frame 18
trt frame 19
trt frame 20
trt frame 21
trt frame 22
trt frame 23
trt frame 24
trt frame 25
trt frame 26
trt frame 27
trt frame 28
trt frame 29
trt frame 30
trt frame 31
trt frame 32
trt frame 33
trt frame 34
trt frame 35
trt frame 36
trt frame 37
trt frame 38
trt frame 39
trt frame 40
trt frame 41
trt frame 42
trt frame 43
trt frame 44
trt frame 45
trt frame 46
trt frame 47
trt frame 48
trt frame 49
trt frame 50
trt frame 51
trt frame 52
trt frame 53
trt frame 54
trt frame 55
trt frame 56
trt frame 57
trt frame 58
trt frame 59
trt frame 60
trt frame 61
trt frame 62
trt frame 63
trt frame 64
trt frame 65
trt frame 66
trt frame 67
trt frame 68
trt frame 69
trt frame 70
trt frame 71
trt frame 72
trt frame 73
trt frame 74
trt frame 75
trt frame 76
trt frame

In [5]:
# malva
transform = transforms.Compose([
    transforms.ToTensor()
])
amplification = 20
amp_f_tensor = torch.tensor(
            [[float(amplification)]], dtype=torch.float, device=device)

torch.cuda.empty_cache()
for i in range(377):
    print("cuda frame", i)
    frame_a = cv2.imread('data/examples/malva_frames/malva' + '{:03d}'.format(i) + '.jpg')[...,::-1]
    frame_a = torch.unsqueeze(transform(to_pil_image(frame_a)).to(device), 0)
    frame_b = cv2.imread('data/examples/malva_frames/malva' + '{:03d}'.format(i+1) + '.jpg')[...,::-1]
    frame_b = torch.unsqueeze(transform(to_pil_image(frame_b)).to(device), 0)
    pred, _, _ = m.forward(
        frame_a.to(device),
        frame_b.to(device),
        amp_f_tensor.reshape(1, 1).to(device))
    pred = pred[0].permute(1,2,0).detach().cpu().numpy()[...,::-1]
    file_path = 'data/examples/malva_frames_cuda/malva' + '{:03d}'.format(i) + 'cuda.jpg'
    cv2.imwrite(file_path, 255*pred)

torch.cuda.empty_cache()
m_trt = torch.jit.load("model_trt_640_360.ts")
for i in range(377):
    print("trt frame", i)
    frame_a = cv2.imread('data/examples/malva_frames/malva' + '{:03d}'.format(i) + '.jpg')[...,::-1]
    frame_a = torch.unsqueeze(transform(to_pil_image(frame_a)).to(device), 0)
    frame_b = cv2.imread('data/examples/malva_frames/malva' + '{:03d}'.format(i+1) + '.jpg')[...,::-1]
    frame_b = torch.unsqueeze(transform(to_pil_image(frame_b)).to(device), 0)
    pred, _, _ = m_trt.forward(
        frame_a.to(device),
        frame_b.to(device),
        amp_f_tensor.reshape(1, 1).to(device))
    pred = pred[0].permute(1,2,0).detach().cpu().numpy()[...,::-1]
    file_path = 'data/examples/malva_frames_trt/malva' + '{:03d}'.format(i) + 'trt.jpg'
    cv2.imwrite(file_path, 255*pred)

cuda frame 0
cuda frame 1
cuda frame 2
cuda frame 3
cuda frame 4
cuda frame 5
cuda frame 6
cuda frame 7
cuda frame 8
cuda frame 9
cuda frame 10
cuda frame 11
cuda frame 12
cuda frame 13
cuda frame 14
cuda frame 15
cuda frame 16
cuda frame 17
cuda frame 18
cuda frame 19
cuda frame 20
cuda frame 21
cuda frame 22
cuda frame 23
cuda frame 24
cuda frame 25
cuda frame 26
cuda frame 27
cuda frame 28
cuda frame 29
cuda frame 30
cuda frame 31
cuda frame 32
cuda frame 33
cuda frame 34
cuda frame 35
cuda frame 36
cuda frame 37
cuda frame 38
cuda frame 39
cuda frame 40
cuda frame 41
cuda frame 42
cuda frame 43
cuda frame 44
cuda frame 45
cuda frame 46
cuda frame 47
cuda frame 48
cuda frame 49
cuda frame 50
cuda frame 51
cuda frame 52
cuda frame 53
cuda frame 54
cuda frame 55
cuda frame 56
cuda frame 57
cuda frame 58
cuda frame 59
cuda frame 60
cuda frame 61
cuda frame 62
cuda frame 63
cuda frame 64
cuda frame 65
cuda frame 66
cuda frame 67
cuda frame 68
cuda frame 69
cuda frame 70
cuda frame 71
cu

trt frame 0
trt frame 1
trt frame 2
trt frame 3
trt frame 4
trt frame 5
trt frame 6
trt frame 7
trt frame 8
trt frame 9
trt frame 10
trt frame 11
trt frame 12
trt frame 13
trt frame 14
trt frame 15
trt frame 16
trt frame 17
trt frame 18
trt frame 19
trt frame 20
trt frame 21
trt frame 22
trt frame 23
trt frame 24
trt frame 25
trt frame 26
trt frame 27
trt frame 28
trt frame 29
trt frame 30
trt frame 31
trt frame 32
trt frame 33
trt frame 34
trt frame 35
trt frame 36
trt frame 37
trt frame 38
trt frame 39
trt frame 40
trt frame 41
trt frame 42
trt frame 43
trt frame 44
trt frame 45
trt frame 46
trt frame 47
trt frame 48
trt frame 49
trt frame 50
trt frame 51
trt frame 52
trt frame 53
trt frame 54
trt frame 55
trt frame 56
trt frame 57
trt frame 58
trt frame 59
trt frame 60
trt frame 61
trt frame 62
trt frame 63
trt frame 64
trt frame 65
trt frame 66
trt frame 67
trt frame 68
trt frame 69
trt frame 70
trt frame 71
trt frame 72
trt frame 73
trt frame 74
trt frame 75
trt frame 76
trt frame

In [6]:
# turtle
transform = transforms.Compose([
    transforms.ToTensor()
])
amplification = 20
amp_f_tensor = torch.tensor(
            [[float(amplification)]], dtype=torch.float, device=device)

torch.cuda.empty_cache()
for i in range(301):
    print("cuda frame", i)
    frame_a = cv2.imread('data/examples/turtle_frames/turtle' + '{:03d}'.format(i) + '.jpg')[...,::-1]
    frame_a = torch.unsqueeze(transform(to_pil_image(frame_a)).to(device), 0)
    frame_b = cv2.imread('data/examples/turtle_frames/turtle' + '{:03d}'.format(i+1) + '.jpg')[...,::-1]
    frame_b = torch.unsqueeze(transform(to_pil_image(frame_b)).to(device), 0)
    pred, _, _ = m.forward(
        frame_a.to(device),
        frame_b.to(device),
        amp_f_tensor.reshape(1, 1).to(device))
    pred = pred[0].permute(1,2,0).detach().cpu().numpy()[...,::-1]
    file_path = 'data/examples/turtle_frames_cuda/turtle' + '{:03d}'.format(i) + 'cuda.jpg'
    cv2.imwrite(file_path, 255*pred)

torch.cuda.empty_cache()
m_trt = torch.jit.load("model_trt_640_360.ts")
for i in range(301):
    print("trt frame", i)
    frame_a = cv2.imread('data/examples/turtle_frames/turtle' + '{:03d}'.format(i) + '.jpg')[...,::-1]
    frame_a = torch.unsqueeze(transform(to_pil_image(frame_a)).to(device), 0)
    frame_b = cv2.imread('data/examples/turtle_frames/turtle' + '{:03d}'.format(i+1) + '.jpg')[...,::-1]
    frame_b = torch.unsqueeze(transform(to_pil_image(frame_b)).to(device), 0)
    pred, _, _ = m_trt.forward(
        frame_a.to(device),
        frame_b.to(device),
        amp_f_tensor.reshape(1, 1).to(device))
    pred = pred[0].permute(1,2,0).detach().cpu().numpy()[...,::-1]
    file_path = 'data/examples/turtle_frames_trt/turtle' + '{:03d}'.format(i) + 'trt.jpg'
    cv2.imwrite(file_path, 255*pred)

cuda frame 0
cuda frame 1
cuda frame 2
cuda frame 3
cuda frame 4
cuda frame 5
cuda frame 6
cuda frame 7
cuda frame 8
cuda frame 9
cuda frame 10
cuda frame 11
cuda frame 12
cuda frame 13
cuda frame 14
cuda frame 15
cuda frame 16
cuda frame 17
cuda frame 18
cuda frame 19
cuda frame 20
cuda frame 21
cuda frame 22
cuda frame 23
cuda frame 24
cuda frame 25
cuda frame 26
cuda frame 27
cuda frame 28
cuda frame 29
cuda frame 30
cuda frame 31
cuda frame 32
cuda frame 33
cuda frame 34
cuda frame 35
cuda frame 36
cuda frame 37
cuda frame 38
cuda frame 39
cuda frame 40
cuda frame 41
cuda frame 42
cuda frame 43
cuda frame 44
cuda frame 45
cuda frame 46
cuda frame 47
cuda frame 48
cuda frame 49
cuda frame 50
cuda frame 51
cuda frame 52
cuda frame 53
cuda frame 54
cuda frame 55
cuda frame 56
cuda frame 57
cuda frame 58
cuda frame 59
cuda frame 60
cuda frame 61
cuda frame 62
cuda frame 63
cuda frame 64
cuda frame 65
cuda frame 66
cuda frame 67
cuda frame 68
cuda frame 69
cuda frame 70
cuda frame 71
cu

trt frame 0
trt frame 1
trt frame 2
trt frame 3
trt frame 4
trt frame 5
trt frame 6
trt frame 7
trt frame 8
trt frame 9
trt frame 10
trt frame 11
trt frame 12
trt frame 13
trt frame 14
trt frame 15
trt frame 16
trt frame 17
trt frame 18
trt frame 19
trt frame 20
trt frame 21
trt frame 22
trt frame 23
trt frame 24
trt frame 25
trt frame 26
trt frame 27
trt frame 28
trt frame 29
trt frame 30
trt frame 31
trt frame 32
trt frame 33
trt frame 34
trt frame 35
trt frame 36
trt frame 37
trt frame 38
trt frame 39
trt frame 40
trt frame 41
trt frame 42
trt frame 43
trt frame 44
trt frame 45
trt frame 46
trt frame 47
trt frame 48
trt frame 49
trt frame 50
trt frame 51
trt frame 52
trt frame 53
trt frame 54
trt frame 55
trt frame 56
trt frame 57
trt frame 58
trt frame 59
trt frame 60
trt frame 61
trt frame 62
trt frame 63
trt frame 64
trt frame 65
trt frame 66
trt frame 67
trt frame 68
trt frame 69
trt frame 70
trt frame 71
trt frame 72
trt frame 73
trt frame 74
trt frame 75
trt frame 76
trt frame

In [13]:

transform = transforms.Compose([
    transforms.ToTensor()
])
amplification = 20
amp_f_tensor = torch.tensor(
            [[float(amplification)]], dtype=torch.float, device=device)

frame_a = cv2.imread('data/examples/malva_frames/malva020.jpg')[...,::-1]
frame_a = torch.unsqueeze(transform(to_pil_image(frame_a)).to(device), 0)
frame_b = cv2.imread('data/examples/malva_frames/malva024.jpg')[...,::-1]
frame_b = torch.unsqueeze(transform(to_pil_image(frame_b)).to(device), 0)
pred, _, _ = m.forward(
    frame_a.to(device),
    frame_b.to(device),
    amp_f_tensor.reshape(1, 1).to(device))
pred = pred[0].permute(1,2,0).detach().cpu().numpy()[...,::-1]
file_path = 'malva_05fps.jpg'
cv2.imwrite(file_path, 255*pred)

True